In [22]:
from Bio.Seq import Seq

In [29]:
%%timeit -n 10
myseq = Seq('AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGC'*10000)
countA,countG,countC,countT = (myseq.count(x) for x in ['A','G','C','T'])

6.5 ms ± 483 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
%%timeit -n 10
myseq = 'AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGC'*10000
countA,countG,countC,countT = (myseq.count(x) for x in ['A','G','C','T'])

8.43 ms ± 2.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
myseq = Seq('TGAGCCTTATTGCTCAGTTGCAACACTAAAAGGGACGTACTTTTTTGCTGGCTGATGTGGTAGACAGAGGGGTTTGAACGGTCAGGGGCGCGCAGTATACAGAGGAATAGTCCTTAAAGCCGCAGTCAATTCCTCTCAAGCAGCCTCATAGAAGTCTGCCGGCGGGAAGAAGATATTTCATGCTATGTTCCAGATTATTAGGCTCCATATTGAACGATGATTAACGTTGTCTTGCCACTATTGACATAATTCTCTAATTTATTTTTGGATCGCACAAGTGGCAAGTTTGAAGTATTGTCACCGTCGAGAACTCGGTAGCAAATACGTCCAATCGGAGACTTCCCCGGATCGTCATCATGGATTATTGGGGGGCCAAAAGACTTAAACGGGCGCCTCCAGCGATCAGCGCCTGACCGTTTAATATGTATATTCGATCGCGCCTCGCGACCCGACCCGTTGCGCCAATGCACCATAAAGCCGGGGCACCAGTACCGATTTGGATGAAGTGTAATTTGGCTCTTGCATCAAAGAAGGCCAAGGTGAGATCGTCAGTAGGAACTTTCTAAGTTTTAATTACTTATTGGTGCGCAAGTCTAATCGCTTGAACCTATGACTCACAATTCCGGCAAGCGTATGTTCTCGCGTAAGGGGGAGATTTTTCTCAGAGACCCTCAGTCAGATGACTCACGAGACAGCCGTATTCTCGTGTGGCCTGGGCAGGAGCAGAACCAGATGTACTGCCAGAGAGACAACTTGTAGGGTGGACAAAGACCCGGCGTAGCGCCGCACTTACAGAGTCTCCTTGCCCCGTATGGAGGCATTCTTAGGGTGCGTAAAGAGTGCAGTCGATCAGCACGGTCTTTGGGACCTAAAATGCACGGTGAGTTAAGGGGTCATTCGCTAGAAACGGGACG')
countA,countG,countC,countT = (myseq.count(x) for x in ['A','G','C','T'])
print(countA,countG,countC,countT)

235 242 207 230


### Manual extraction of protein processes

In [103]:
import re
import urllib.request

uniprot_id = 'Q817I4'
url = f'http://www.uniprot.org/uniprot/{uniprot_id}.txt'

with urllib.request.urlopen(url) as f:
    data = f.read().splitlines()

processes = []
for line in data:
    if line[:7] == b'DR   GO':
        interest = str(line[21:])[2:-1]
        if (interest[0] == 'P'):
            processes.append(re.sub(';.*','',interest[2:])) 

print(*processes,sep='\n')

asexual sporulation
sporulation resulting in formation of a cellular spore


### Now using Biopython

In [118]:
from Bio import ExPASy
from Bio import SwissProt
handle = ExPASy.get_sprot_raw('Q817I4') #you can give several IDs separated by commas
                                        #(but I think they all have to be one string)
record = SwissProt.read(handle) # use SwissProt.parse for multiple proteins

processes = []
for line in record.cross_references: # found cross_references attribute using dir(record)
    if line[0] == 'GO':
        processes.append(line[2][2:])
print(*processes,sep='\n')

asexual sporulation
sporulation resulting in formation of a cellular spore


### Genbank search
Genbank can be accessed with Entrez Nucleotide. More Genbank details at https://www.ncbi.nlm.nih.gov/genbank/

In [145]:
from Bio import Entrez

Given = 'Trichoderma \
            2001/09/08 \
            2002/04/02' 
Given = Given.split()
SearchTerm,StartDate,EndDate = Given
Query = f'({SearchTerm}[Organism]) AND ("{StartDate}"[Publication Date] : "{EndDate}"[Publication Date])'


Entrez.email = 'brianjculver@gmail.com'
Search = Entrez.esearch(db='nucleotide', term=Query)
record = Entrez.read(Search)
record['Count']             # Number of records with Organism name published between given dates

'146'

Using Fasta formatted files from Genbank using Entrez.efetch(db,rettype)

In [183]:
from Bio import Entrez
Entrez.email = 'brianjculver@gmail.com'
id_string = 'JX205496 NM_001079732 JX469985 NM_001081821 JQ011276 FJ817486 JX469983 JX317645 JX398977'
id_list = ', '.join(id_string.split())  # series of ids in one string separated by commas
handle = Entrez.efetch(db='nucleotide',id=[id_list],rettype='fasta')
records = handle.read() #returns fasta file in string format
records = records.split('\n>')
lenlist = [len(record) for record in records]
for record in records:
    if len(record) == min(lenlist):    # printing shortest record from list
        print('>'+record)

>JX317645.1 Culex quinquefasciatus neuropeptide F mRNA, complete cds
ATGGCATCCACAAGCAGCAGCAGCAGAATCAACAACAACCGCCATGCCGTCAGGTCATCAGCCTCTTCAG
CGTTCACCCAGCGACTGCTAATCGGCCTGCTGGTCTGCACCCTGGTGCTGGATCTTAGCTGCCTGACCGA
GGCCCGGCCCCAGGACGATCCCACCTCCGTCGCCGAAGCCATCCGACTGCTGCAGGAGCTGGAAACCAAG
CACGCCCAACATGCCCGACCAAGATTCGGAAAACGTGGCTATCTCCAGCCGGCAAGTTACGGCCAGGACG
AACAGGAGCGAAACTATTACAGGATGATTGGCAGGATTCAGCGTTTTCAAGATGAACAGAACGCCGTACT
CAACTAA



Using Bio.SeqIO to handle fasta format

In [189]:
from Bio import SeqIO
from Bio import Entrez
Entrez.email = 'brianjculver@gmail.com'
id_list = "FJ817486, JX069768, JX469983"  # series of ids in one string
handle = Entrez.efetch(db='nucleotide',id=[id_list],rettype='fasta')

records_gen = SeqIO.parse(handle, "fasta") # generator object
records = list(records_gen) #list of SeqRecord objects
lenlist = [len(record.seq) for record in records]
for record in records:
    sequence = record.seq
    if len(sequence) == min(lenlist):      # printing shortest record from list
        print('>'+record.description+'\n'+sequence)

>JX469983.1 Zea mays subsp. mays clone UT3343 G2-like transcription factor mRNA, partial cds
ATGATGTATCATGCGAAGAATTTTTCTGTGCCCTTTGCTCCGCAGAGGGCACAGGATAATGAGCATGCAAGTAATATTGGAGGTATTGGTGGACCCAACATAAGCAACCCTGCTAATCCTGTAGGAAGTGGGAAACAACGGCTACGGTGGACATCGGATCTTCATAATCGCTTTGTGGATGCCATCGCCCAGCTTGGTGGACCAGACAGAGCTACACCTAAAGGGGTTCTCACTGTGATGGGTGTACCAGGGATCACAATTTATCATGTGAAGAGCCATCTGCAGAAGTATCGCCTTGCAAAGTATATACCCGACTCTCCTGCTGAAGGTTCCAAGGACGAAAAGAAAGATTCGAGTGATTCCCTCTCGAACACGGATTCGGCACCAGGATTGCAAATCAATGAGGCACTAAAGATGCAAATGGAGGTTCAGAAGCGACTACATGAGCAACTCGAGGTTCAAAGACAACTGCAACTAAGAATTGAAGCACAAGGAAGATACTTGCAGATGATCATTGAGGAGCAACAAAAGCTTGGTGGATCAATTAAGGCTTCTGAGGATCAGAAGCTTTCTGATTCACCTCCAAGCTTAGATGACTACCCAGAGAGCATGCAACCTTCTCCCAAGAAACCAAGGATAGACGCATTATCACCAGATTCAGAGCGCGATACAACACAACCTGAATTCGAATCCCATTTGATCGGTCCGTGGGATCACGGCATTGCATTCCCAGTGGAGGAGTTCAAAGCAGGCCCTGCTATGAGCAAGTCA


### FASTQ format

In [2]:
from Bio import SeqIO
SeqIO.convert('fastq_test.fastq','fastq','fasta_result.fasta','fasta')

1